# 문용준님 텐서플로우 슬라이드

# Tensorflow 빌딩/실행 구조

In [4]:
#tensorflow 구조
#텐서플로우는 엣지와 노드로 구조화된 그래프로 프로그램이 구성
import tensorflow as tf
a = tf.constant(1)
print(a)
with tf.Session() as sess:
    print(a.eval())
    
#처리 순서: 빌딩구조와 실행구조 (session)
x = tf.constant(35, name='x')
y = tf.Variable(x + 5, name='y')

model = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(model)
    print(session.run(y))
    
#With 구문: close()를 사용 안해도 됨
x = tf.Variable(0, name='x')

model = tf.global_variables_initializer()

with tf.Session() as session:
    for i in range(5):
        session.run(model)
        x = x + 1
        print(session.run(x))
        
#fetch: 1개 실행 예시
with tf.Session() as sess:
    print(tf.add(1,1).eval())

#fetch: 여러개 실행 예시 (Session.run에서 리스트로 여러 개 실행되는 텐서 처리)
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
intpu3 = tf.constant(5.0)

intermed = tf.add(input2, input3)
mul = tf.mul(input1, intermed)

with tf.Session() as sess:
    result = sess.run ()

Tensor("Const_3:0", shape=(), dtype=int32)
1
40
1
2
3
4
5
2


In [6]:
#Tensorboard 실행
a = tf.add(1, 2)
b = tf.mul(a, 3)
c = tf.add(4, 5)
d = tf.mul(c, 6)

with tf.Session() as sess:
    writer = tf.summary.FileWriter("./", sess.graph)
    print(sess.run(d))
    writer.close()
    
#!tensorboard --logdir="./"

54


# Tensorflow 실행 구조

## Tensorflow Graph Class 이해하기

In [4]:
#tf.Graph: Operation과 Tensor들의 집합

#Tensorflow 생성
import tensorflow as tf
x = tf.constant(1.0, name='input')
w = tf.Variable(0.8, name='weight')
y = tf.mul(w,x, name='output')

sess = tf.Session()
summary_writer = tf.summary.FileWriter('./test/', sess.graph)

In [8]:
#tensorflow graph 객체
#텐서플로에서 graph를 직접 접근해서 node들이 생성된 것을 볼 수 있음
graph = tf.get_default_graph()
graph.get_operations()

In [10]:
#Tensorflow: graph op name
for op in graph.get_operations(): print op.name

input
weight/initial_value
weight
weight/Assign
weight/read
output
input_1
weight_1/initial_value
weight_1
weight_1/Assign
weight_1/read
output_1
input_2
weight_2/initial_value
weight_2
weight_2/Assign
weight_2/read
output_2


# Node 확인하기 (순차)

# Model 저장하기 및 불러오기

In [8]:
# X 와 Y 의 상관관계를 분석하는 기초적인 선형 회귀 모델을 만들고 실행해봅니다.

import tensorflow as tf
import numpy as np

In [ ]:
# X 와 Y 의 상관관계를 분석하는 기초적인 선형 회귀 모델을 만들고 실행해봅니다.

import tensorflow as tf
import numpy as np

#input_data = [[1,5,3,7,8,10,12],
#             [5,8,10,3,9,7,1]]
input_data = np.random.randint(10, size=(2,7))
label_data = [[0,0,0,1,0],
             [1,0,0,0,0]
             ]
label_data = np.array(label_data)

INPUT_SIZE = 7
HIDDEN1_SIZE = 10
HIDDEN2_SIZE = 8
CLASSES = 5

Learning_Rate = 0.05

x = tf.placeholder( "float", shape=[None, INPUT_SIZE], name='x')
y_ = tf.placeholder( "float", shape=[None, CLASSES], name='y_')

#Tensor map 적용
tensormap = {x: input_data, y_: label_data}

#Layer1
W_h1 = tf.Variable( tf.truncated_normal(shape=[INPUT_SIZE, HIDDEN1_SIZE]), dtype = tf.float32, name ="W_h1")
b_h1 = tf.Variable( tf.zeros(shape=[HIDDEN1_SIZE]), dtype=tf.float32, name ="b_h1")

#Layer2
W_h2 = tf.Variable( tf.truncated_normal(shape=[HIDDEN1_SIZE, HIDDEN2_SIZE]), dtype = tf.float32, name ="W_h2")
b_h2 = tf.Variable( tf.zeros(shape=[HIDDEN2_SIZE]), dtype=tf.float32, name ="b_h2")

#Output Layer
W_o = tf.Variable( tf.truncated_normal(shape=[HIDDEN2_SIZE, CLASSES]), dtype = tf.float32, name ="W_o")
b_o = tf.Variable( tf.zeros(shape=[CLASSES]), dtype=tf.float32, name ="b_o")

param_list = [W_h1, b_h1, W_h2, b_h2, W_o, b_o]

with tf.name_scope('hidden_layer_1') as h1scope:
    hidden1 = tf.sigmoid( tf.matmul(x, W_h1) + b_h1, name='hidden1')

with tf.name_scope('hidden_layer_2') as h2scope:
    hidden2 = tf.sigmoid( tf.matmul(hidden1, W_h2) + b_h2, name='hidden2')

with tf.name_scope('output_layer') as oscope:
    y = tf.sigmoid( tf.matmul(hidden2, W_o) + b_o, name='y')

#Summary는 로그를 지속적으로 모으기

# 손실 함수를 작성합니다.
cost = tf.reduce_sum(-y_*tf.log(y)-(1-y_)*tf.log(1-y), reduction_indices=1)
cost = tf.reduce_mean( cost )
train = tf.train.GradientDescentOptimizer(Learning_Rate).minimize(cost)

comp_pred = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
acc = tf.reduce_mean(tf.cast(comp_pred, tf.float32))

#Model을 저장합니다.
saver = tf.train.Saver(param_list) #아무것도 넣지 않으면 모두 저장이됨

# 세션을 생성하고 초기화합니다.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    merge = tf.summary.merge_all()
    # 최적화를 1000번 수행합니다.
    for step in range(1000):
        # sess.run 을 통해 train_op 와 cost 그래프를 계산합니다.
        # 이 때, 가설 수식에 넣어야 할 실제값을 feed_dict 을 통해 전달합니다.
        _, loss = sess.run([train, cost], feed_dict=tensormap)
        if step % 100 == 0:
            saver.save(sess, './model/my-model', global_step=step)
            train_writer = tf.summary.FileWriter('./summaries/', sess.graph)
            print("---------")
            print("Step: ", step)
            print("loss: ", loss)
            print("accuracy", acc)

#Model Restore
#Name을 분명히 입력하자
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('./model/my-model-900.meta')
    new_saver.restore(sess, tf.train.latest_checkpoint('./model/'))
    result = sess.run( y, tensormap )
    print(result)

#tensorboard --logdir=./